In [ ]:
# !pip install -q fasttext pandas xlrd langdetect

In [ ]:
from pathlib import Path
import pandas as pd
import fasttext.util
from langdetect import detect
import sklearn.metrics as metrics
import numpy as np
import os
import string
from utils.preprocessing import tokenize

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)

In [ ]:
text = pd.read_csv('data/interim/row_df_fi.csv', index_col=0)['answer']
text.sample(n=10)

In [ ]:
# esco skills
skills = pd.read_csv('data/external/skills_fi.csv')
skills.head()

In [ ]:
model_name = 'cc.fi.300.bin'

# download pretrained fasttext model if it does not exist
if not Path('models', model_name).is_file():
    fasttext.util.download_model('fi', if_exists='ignore')
    Path(model_name).rename(os.path.join('models', model_name))
    
ft = fasttext.load_model('models/cc.fi.300.bin')

In [ ]:
# stack all esco skill vectors
M = np.stack(skills['preferredLabel'].apply(ft.get_sentence_vector))
M.shape

In [ ]:
v = ft.get_sentence_vector('vuorovaikutustaidot')

cos_sim = metrics.pairwise.cosine_similarity(M, v.reshape(1, -1))

sim = pd.DataFrame({'skill': skills['preferredLabel'], 'similarity': cos_sim.reshape(-1)})
sim[sim['similarity'] > 0.5].sort_values('similarity', ascending=False).reset_index(drop=True)[:10]

In [ ]:
THRESHOLD = 0.7

# tässä käydään läpi jokaisen vastauksen jokainen sana ja pyritään tunnistamaan taidot muun tekstin joukosta
# verrataan jokaista sanaa esco-osaamisten vektoreihin ja esim. luokitellaan sana osaamiseksi jos sille löytyy ESCO-osaaminen jonka kanssa samankaltaisuus >= `THRESHOLD`

for doc in text[:10]:
    for w in tokenize(doc):
        skill_vec = ft.get_sentence_vector(w)

        similarity = metrics.pairwise.cosine_similarity(M, skill_vec.reshape(1, -1))

        sim_df = pd.DataFrame({'similarity': similarity.reshape(-1)}, index=skills['preferredLabel'])

        similar = sim_df[sim_df['similarity'] > THRESHOLD]

        s = [t.similarity for t in similar.itertuples()]

        print(w)

        for tup in list(similar.itertuples())[:5]:
            print(f'\t {tup.Index} {tup.similarity}')

    print('#' * 80)



Tämä menetelmä taitojen tunnistamiseen näyttäisi olevan puutteellinen (ainakin) kahdella tavalla: malli tarttuu liikaa syntaktisiin piirteisiin (esim. kehittäminen, kanssa, tai liittyviä) ja tekstissä olevat taidot eivät yleensä ole yksittäisiä sanoja joten niitä ei voi tunnistaa tällä menetelmällä.

Tätä menetelmää voisi periaatteessa soveltaa useaan peräkkäiseen sanaan, mutta aikaisemmat kokeilut (tällä datalla) ovat näyttäneet että näin saadaan paljon osumia esco-taidoista mutta ne eivät ole millään tavalla relevantteja.